In [ ]:
import pandas as pd
import numpy as np
import os
import datetime
import matplotlib.pyplot as plt
#dir = os.getcwd()
dir = "C:\\Users\\fbeth\\OneDrive\\Ablage\\"
#dir = "C:\\Users\\u51051\Downloads"
for file in os.listdir(dir):
    if file.endswith('RWD2.csv') and file.__contains__('2022-12'):
        file = os.path.join(dir, file)
        break

print(file)

______
### import .csv into DataFrame and remove whitespace and brackets from column names to use attribute access of df further below (autocomplete etc)

In [ ]:
df = pd.read_csv(file, delimiter=';')#.head(200)
df.PID.replace(r"\s+|\[|\]|\(|\)|\.|\:|\-|\/|\#|08|8C|51", '', regex=True, inplace=True)
df=df[~df.PID.str.contains('Cell')]

In [ ]:
def show_2d():
    df.index = df.SECONDS
    df.index = pd.Timestamp(datetime.date.today()) + pd.TimedeltaIndex(df.index, unit='s')
    df.index.name = 'Time'
    plt.figure(figsize=(10,8))
    #df=df["2022-08-30 23:12:00.6":"2022-08-30 23:26:00.6"]
    plt.scatter(x=df.LONGTITUDE, y=df.LATITUDE,s=1)
    plt.show()

#show_2d()

_____________
### reshape table from purely line based to value columns with pivot
a few lines will be merged

In [ ]:
df = df.pivot_table( index = 'SECONDS', 
                        columns='PID', 
                        values='VALUE')
df.columns.name = None

__________________
### convert seconds into Timestamp-Index

In [ ]:

df.index = pd.Timestamp(datetime.date.today()) + pd.TimedeltaIndex(df.index, unit='s')
df.index.name = 'Time'

___
resample to merge some lines

add own power

In [ ]:
df.index
df=df.resample('500ms').mean()
df["Power"]=df.BMSBatteryVoltage*df.BMSHVBatteryCurrent/1000
hz=2  # frequency used later on 

_______________
### fill the gaps
not always needed if already resampled data???

In [ ]:
df.interpolate(method='linear', order=2, inplace=True, limit_direction='both')
df.columns

### translate to english if needed

In [ ]:
df.rename(columns = {"Geschwindigkeit":"Vehiclespeed"}, inplace=True)
df.rename(columns = {"zurückgelegteStrecke":"Distancetravelled"}, inplace=True)
df.rename(columns = {"HöheGPS":"AltitudeGPS"}, inplace=True)
df.columns

In [ ]:
#df=df[1400:3400]

#df.SpeedGPS.plot()  #df.Vehiclespeed.plot()
#df.PTCWaterpumppower.multiply(0.01).plot()
#df.Tripcomputer.plot()
#df.PTCHeaterCurrent.plot()
#df.BMSHVBatteryTemperatureAverage.multiply(1).plot()
#df.Distancetravelled.multiply(5).plot() #df.Distancetravelled.multiply(4).plot()
#df.HVACInteriortemperature.multiply(1).plot(figsize=(16,12), grid=True)

fig, ax1 = plt.subplots()


ax1.plot(df.Vehiclespeed, label="Velocity")
ax1.plot(df.Power, label="Power")

ax1.plot(df.BMSMaximumdynamicchargecurrentlimit.multiply(df.BMSBatteryVoltage/-1000),  label="Max recu in kW")
ax1.plot(df.BMSMaximumdynamicchargecurrentlimit.multiply(-1), label="Max recu in Amp")
ax2 = ax1.twinx()
ax2.plot(df.SoCtrue, color="yellow", linestyle='dashed',label="Soc" )
ax2.plot(df.BMSHVBatteryTemperatureAverage, color='lightblue', linestyle="dashed", label="Temperature")

ax1.legend(loc='upper left')
ax2.legend(loc='upper right')
ax1.grid()
ax2.set_ylim([-20,80])

fig.set_size_inches(12, 7)
plt.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.1)

plt.show()

In [ ]:
#df.to_excel("michael_Leihcan RWD.xlsx")
stop

In [ ]:
df.EVInstantEnergyConsumptionkWh100km.plot(figsize=(15,11), grid=True)

now try to find the laps by evaluation starts from 0 kmh or delta v:

In [ ]:
np.set_printoptions(precision=2)

def find_test_ranges (start_v, end_v):
    finds = []
    found=False; found_end=False
    for i in range(0,df.shape[0]-1,1):
        if df.Vehiclespeed[i] >= start_v and df.Vehiclespeed[i+1] < start_v:
            start=i
            found=True
    
        if found==True and df.Vehiclespeed[i] < end_v and df.Vehiclespeed[i-1] >= end_v:
            end=i
            found_end=True

        if found_end:
            finds.append({"start":start,"end":end})
            found=False; found_end=False

    return finds

def find_laps (zeroSecs):
    finds = []
    found=False; found_end=False
    for i in range(0,df.shape[0]-1,1):
        if found == False and df.Vehiclespeed[i] == 0 and df.Vehiclespeed[i+1] > 0:
            start=i
            found=True
    
        if found==True and df.Vehiclespeed[i] == 0 and df.Vehiclespeed[i+zeroSecs] == 0:
            end=i
            found_end=True

        if found_end:
            finds.append({"start":start,"end":end})
            found=False; found_end=False

    return finds
        
def find_speed_ranges (speed, max_diff):
    finds = []
    found=False; found_end=False
    for i in range(0,df.shape[0]-1,1):
        if df.Vehiclespeed[i] <= speed and df.Vehiclespeed[i+1] > speed:
            start=i
            found=True
    
        if found==True and df.Vehiclespeed[i] < speed-max_diff:
            end=i
            found_end=True

        if found_end:
            finds.append({"start":start,"end":end})
            found=False; found_end=False

    return finds
 
def get_diffs(finds):
    diffs=[]
    for f in finds:
        #print(f)
        diffs.append([df.Distancetravelled[f["end"]]-df.Distancetravelled[f["start"]],
        (f["end"]-f["start"])/2.0,
        #df.BMSHVBatterySoChighres[f["end"]]-df.BMSHVBatterySoChighres[f["start"]],
        df.AltitudeGPS[f["end"]]-df.AltitudeGPS[f["start"]],
        #df.Battenergy[ends[y]]-df.Battenergy[starts[y]],
        df.EVInstantEnergyConsumptionkWh100km[f["start"]:f["end"]].mean(),
        df.Power[f["start"]:f["end"]].sum()/(hz*3600)] )

    return diffs
    

In [ ]:
#fs=find_laps(1)
fs=find_speed_ranges(192,1)
print(fs)
for i in fs:   
    #print(i)
    print(df.Distancetravelled[i["start"]],
        df.Tripcomputer[i["start"]])
        #df.BMSHVBatterySoChighres[i["start"]],
        #df.Battenergy[i["start"]])
  #  print(df.Vehiclespeed[i["start"]:i["end"]].describe())
print("\n--diffs--")
[[round(i,3) for i in d] for d in get_diffs(fs)]


der Verbrauch aus den summierten voltages

In [ ]:

print([[round(100*d[4]/d[0],3) ] for d in get_diffs(fs)])

In [ ]:
s=[f["start"] for f in fs]
e=[f["end"] for f in fs]
df["starts"] = df.iloc[s].Vehiclespeed
df["ends"] = df.iloc[e].Vehiclespeed
plt.figure(figsize=(20,16))
plt.scatter(df.index,df["starts"],c='g')
plt.scatter(df.index,df["ends"],c='r')

plt.plot(df.index,df.Vehiclespeed)

plt.show()

In [ ]:
df.BMSHVBatteryTemperatureAverage.plot(grid=True, figsize=(12,8), ylim=(10,32))

In [ ]:
df.EVInstantEnergyConsumptionkWh100km.rolling(30).mean().plot(figsize=(11,8),ylim=(-30,120),grid=True)

In [ ]:
for i in fs:
    print(df.Power[i["start"]:i["end"]].sum()/(hz * 3600))
    print(df.HVEVBatteryPower[i["start"]:i["end"]].sum()/(hz * 3600))
    print(df.PTCWaterpumppower[i["start"]:i["end"]].sum()/(hz * 3600000))
    print("-")


In [ ]:
from winsound import Beep
Beep(1000,1000)
df.C6HVCHRGREnergyturnovertotal.min
df.C6HVCHRGREnergyturnovertotal.max
#df.to_csv('C:\\Users\\Falco\\OneDrive\\Ablage\\TaycanMeerbusch.csv')

In [ ]:
from matplotlib.pyplot import ylim


df.Tripcomputer.plot(ylim=(0,40), figsize=(10,9), grid=True)

In [ ]:
dfr=df[pd.to_datetime("11:15"):pd.to_datetime("11:24")]
dfr.HVEVBatteryPower.rolling(10).mean().plot(label='HVEVBatteryPower rolling mean');
#df.HVEVBatteryPower.ewm(span=20).mean().plot(figsize=(11,8))
#df.AltitudeGPS.ewm(span=20).mean().plot(figsize=(11,8))
#dfr.Tripcomputer.plot( color = (1,0,1), label = 'Tripcomputer kWh/100km');
dfr.Vehiclespeed.plot( color = (0,1,0), label = 'Vehicle speed km/h');
#dfr.Distancetravelled.plot( color = (0,0,1), label = 'Distance travelled km');
dfr.EVInstantEnergyConsumptionkWh100km.rolling(10).mean().plot(figsize=(11,8),ylim=(-10,30),grid=True)
plt.legend()
plt.show()


In [ ]:
df.HVEVBatteryPower.plot(kind='hist', bins=np.arange(-270,90,1), figsize=(12,6), xlim=(-90,90))
plt.legend()

In [ ]:
df.filter(regex="51E|Geschw|kWh").rolling(60).mean().plot(ylim=(-10,150), figsize=(15,10))

testing attribute access and renaming value

In [ ]:
#df.Vehiclespeed.plot(label='Vehicle Speed')
#df.rename(columns = {'Vehiclespeed':'speed'}, inplace = True)
#df.Vehiclespeed.plot(figsize=(11,8), label='Vehicle Speed')
#plt.legend()

#df.loc[:,df.AbsolutepedalpositionD]



In [ ]:
df.columns

In [ ]:

ax = df.EVInstantEnergyConsumptionkWh100km[200:36000].rolling(200).mean().plot(ylim=(0,35),label='Tripcomputer', figsize=(15,10))
#ax.set_xticklabels(, rotation=45)
df.Vehiclespeed[200:36000].plot(figsize=(10,5), grid=True, secondary_y=True,ax=ax, label='Vehiclespeed')
ax.legend(loc='upper right')

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(df.Vehiclespeed, color = (0,0,0), linewidth = 1, alpha = .9, label = 'Speed');
ax.plot(df.Tripcomputer, color = (1,0,1), label = 'Tripcomputer kWh/100km');
ax.set_title('Taycan consumption')
ax.set_xlabel('Time')
ax.set_ylabel('consump')
ax.secondary_yaxis=True;
ax.legend(loc='upper right');

In [ ]:
df.Distancetravelled.to_frame()

In [ ]:
temps=df.filter(regex="Cooling|ryT")
print(temps.columns)
ax= temps.rolling(30).mean().plot(figsize=(15,10),ylim=(0,40))
df.PTCWaterpumppower.rolling(30).mean().plot(figsize=(10,6),ylim=(0,40), linewidth=2, alpha=.2, grid=True, label="AC", secondary_y=True, ax=ax)
plt.legend()


In [ ]:
df.apply(pd.Series.max)-df.apply(pd.Series.min)

In [ ]:
fig, ax = plt.subplots()
import matplotlib.dates as md

ax.plot(df.filter(regex="SoC"))

#ax.xaxis.set_major_locator(md.MinuteLocator(interval = 5))
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
plt.setp(ax.xaxis.get_majorticklabels(), rotation = 90)
plt.show()

In [ ]:
df.filter(regex="ySo").plot(figsize=(10,6),ylim=(20,60))